# Feature Engineering e Dados Curados

Este notebook tem como propósito realizar a engenharia de atributos completa a partir dos dados processados, garantindo que o conjunto final de variáveis seja coerente, numerificado e adequado para aprendizado supervisionado. Serão aplicadas as transformações necessárias para resolver problemas de cardinalidade, remover colunas irrelevantes e assegurar que treino e teste estejam alinhados de forma robusta. 

Ao final, os arquivos serão salvos em `data/curated/`, prontos para rastreamento, versionamento e uso em experimentos de modelagem.


## Diagnóstico de Cardinalidade

Esta etapa tem como objetivo analisar todas as colunas não numéricas presentes nos dados de treino e teste, identificando quantos valores únicos cada uma possui. O foco é antecipar possíveis problemas de alta cardinalidade que possam gerar sparsidade excessiva ou explosão de features no momento da codificação.

Será criado um relatório completo com a cardinalidade de cada variável categórica, permitindo decisões fundamentadas sobre agrupamento, transformação ou exclusão. O resultado final desta célula técnica é um arquivo `.csv` salvo em `data/curated/`, que servirá como referência rastreável para as próximas etapas de feature engineering.


In [5]:
# ETAPA: Diagnóstico de Cardinalidade

"""
Esta etapa carrega os datasets processados, identifica todas as colunas não numéricas
e calcula a cardinalidade de cada uma, criando um relatório detalhado.
O objetivo é mapear variáveis com alta cardinalidade que precisem de agrupamento ou tratamento específico.
O relatório final será salvo em 'data/curated/cardinality_report.csv' para rastreabilidade.
"""

import os
import pandas as pd
from tqdm import tqdm

# 1️⃣ Validar diretório de trabalho
print("Current Working Directory:", os.getcwd())

# 2️⃣ Definir paths
TRAIN_PATH = 'data/processed/train_clean.csv'
TEST_PATH = 'data/processed/test_clean.csv'

# 3️⃣ Carregar datasets
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

# 4️⃣ Identificar colunas não numéricas
train_non_numeric = train_df.select_dtypes(include=['object']).columns.tolist()
test_non_numeric = test_df.select_dtypes(include=['object']).columns.tolist()
print("\nColunas não numéricas (treino):", train_non_numeric)
print("Colunas não numéricas (teste):", test_non_numeric)

# 5️⃣ Diagnóstico de cardinalidade
cardinality_info = []

print("\nCalculando cardinalidade por coluna com barra de progresso...")

for col in tqdm(train_non_numeric, desc="Treino"):
    unique_vals = train_df[col].nunique(dropna=True)
    cardinality_info.append({'Set': 'Train', 'Column': col, 'UniqueValues': unique_vals})

for col in tqdm(test_non_numeric, desc="Teste"):
    unique_vals = test_df[col].nunique(dropna=True)
    cardinality_info.append({'Set': 'Test', 'Column': col, 'UniqueValues': unique_vals})

# 6️⃣ Criar DataFrame de relatório
cardinality_df = pd.DataFrame(cardinality_info)

print("\nResumo das 20 primeiras linhas:")
print(cardinality_df.head(20))

# 7️⃣ Salvar relatório para rastreabilidade
os.makedirs('data/curated', exist_ok=True)
cardinality_df.to_csv('data/curated/cardinality_report.csv', index=False)
print("\nRelatório salvo em: data/curated/cardinality_report.csv")


Current Working Directory: /workspace

Colunas não numéricas (treino): ['ID', 'Customer_ID', 'Month', 'Name', 'SSN', 'Occupation', 'Num_of_Loan', 'Type_of_Loan', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit', 'Credit_Mix', 'Credit_History_Age', 'Payment_of_Min_Amount', 'Payment_Behaviour', 'Credit_Score']
Colunas não numéricas (teste): ['Num_of_Loan', 'Type_of_Loan', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit', 'Credit_History_Age', 'Payment_of_Min_Amount']

Calculando cardinalidade por coluna com barra de progresso...


Teste: 100%|██████████| 6/6 [00:00<00:00, 900.84it/s]


Resumo das 20 primeiras linhas:
      Set                  Column  UniqueValues
0   Train                      ID        100000
1   Train             Customer_ID         12500
2   Train                   Month             8
3   Train                    Name         10139
4   Train                     SSN         12501
5   Train              Occupation            16
6   Train             Num_of_Loan           434
7   Train            Type_of_Loan          6260
8   Train  Num_of_Delayed_Payment           749
9   Train    Changed_Credit_Limit          4384
10  Train              Credit_Mix             4
11  Train      Credit_History_Age           404
12  Train   Payment_of_Min_Amount             3
13  Train       Payment_Behaviour             7
14  Train            Credit_Score             3
15   Test             Num_of_Loan           263
16   Test            Type_of_Loan          6260
17   Test  Num_of_Delayed_Payment           443
18   Test    Changed_Credit_Limit          3927
19   Te

## Tratamento de Alta Cardinalidade

Esta etapa tem como objetivo reduzir a dimensionalidade excessiva causada por variáveis categóricas com muitos valores únicos. Serão removidas colunas que são identificadores diretos e não contribuem para aprendizado supervisionado. Além disso, colunas com cardinalidade muito alta serão agrupadas ou transformadas em faixas, quando aplicável, para evitar sparsidade excessiva após a codificação.

O resultado esperado é um conjunto de dados mais compacto, coerente e alinhado entre treino e teste, garantindo que o processamento posterior gere um número controlado de features, mantendo a capacidade preditiva do modelo.


## Coerção de Tipos, Binning Seguro e Padronização

Esta etapa executa de forma consolidada a coerção de tipos para garantir que colunas com dados inconsistentes possam ser transformadas em faixas numéricas de forma segura. Além disso, realiza a padronização da coluna 'Month' apenas quando ela estiver presente, evitando falhas no pipeline. O resultado esperado é um conjunto de dados limpo, com variáveis agrupadas ou convertidas para reduzir a cardinalidade, mantendo a coerência entre treino e teste.


In [8]:
# 🔧 ETAPA: Coerção de Tipos, Binning Seguro e Padronização de 'Month'

"""
Executa:
1) Exclusão de identificadores.
2) Coerção de tipos numéricos com tratamento de strings.
3) Binning controlado para alta cardinalidade.
4) Padronização segura da coluna 'Month'.
Fluxo rastreável, alinhado ao PROTOCOLO V5.4.
"""

import os
import pandas as pd
import numpy as np
from tqdm import tqdm

# 1️⃣ Validar CWD
print("Current Working Directory:", os.getcwd())

# 2️⃣ Paths
TRAIN_PATH = 'data/processed/train_clean.csv'
TEST_PATH = 'data/processed/test_clean.csv'

# 3️⃣ Carregar datasets
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

# 4️⃣ Remover identificadores
drop_cols = ['ID', 'Customer_ID', 'Name', 'SSN']
train_df.drop(columns=drop_cols, errors='ignore', inplace=True)
test_df.drop(columns=drop_cols, errors='ignore', inplace=True)

print("\nColunas removidas:", drop_cols)

# 5️⃣ Coerção de tipos e binning para Num_of_Loan
if 'Num_of_Loan' in train_df.columns:
    for df in [train_df, test_df]:
        df['Num_of_Loan'] = pd.to_numeric(df['Num_of_Loan'], errors='coerce')

    for df in [train_df, test_df]:
        df['Num_of_Loan_Bin'] = pd.cut(
            df['Num_of_Loan'],
            bins=[0,1,2,5,10,20,50,100,500,1000],
            labels=False
        )

    train_df.drop(columns=['Num_of_Loan'], inplace=True)
    test_df.drop(columns=['Num_of_Loan'], inplace=True)
    print("\nBinning aplicado: Num_of_Loan")

# 6️⃣ Outras colunas de alta cardinalidade
for col in ['Changed_Credit_Limit', 'Num_of_Delayed_Payment', 'Credit_History_Age']:
    if col in train_df.columns:
        for df in [train_df, test_df]:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            df[col + '_Bin'] = pd.qcut(
                df[col].rank(method="first"),
                q=10 if col != 'Credit_History_Age' else 5,
                labels=False,
                duplicates='drop'
            )
        train_df.drop(columns=[col], inplace=True)
        test_df.drop(columns=[col], inplace=True)
        print(f"Binning aplicado: {col}")

# 7️⃣ Padronização segura de 'Month'
month_map = {
    'January':1, 'February':2, 'March':3, 'April':4, 'May':5, 'June':6,
    'July':7, 'August':8, 'September':9, 'October':10, 'November':11, 'December':12
}

for df_name, df in [('Treino', train_df), ('Teste', test_df)]:
    if 'Month' in df.columns:
        df['Month_Num'] = df['Month'].map(month_map)
        df.drop(columns=['Month'], inplace=True, errors='ignore')
        print(f"Coluna 'Month' padronizada e removida em: {df_name}")
    else:
        print(f"Coluna 'Month' não encontrada em: {df_name}")

print("\nColunas finais após coerção, binning e padronização:")
print(train_df.columns.tolist())

print("\nAmostra treino (head 20):")
print(train_df.head(20))

# 8️⃣ Salvar versão intermediária
os.makedirs('data/curated', exist_ok=True)
train_df.to_csv('data/curated/train_feature_engineered.csv', index=False)
test_df.to_csv('data/curated/test_feature_engineered.csv', index=False)

print("\nArquivos salvos em: data/curated/train_feature_engineered.csv e data/curated/test_feature_engineered.csv")


Current Working Directory: /workspace

Colunas removidas: ['ID', 'Customer_ID', 'Name', 'SSN']

Binning aplicado: Num_of_Loan
Binning aplicado: Changed_Credit_Limit
Binning aplicado: Num_of_Delayed_Payment
Binning aplicado: Credit_History_Age
Coluna 'Month' padronizada e removida em: Treino
Coluna 'Month' não encontrada em: Teste

Colunas finais após coerção, binning e padronização:
['Age', 'Occupation', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Type_of_Loan', 'Delay_from_due_date', 'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt', 'Credit_Utilization_Ratio', 'Payment_of_Min_Amount', 'Total_EMI_per_month', 'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance', 'Credit_Score', 'Num_of_Loan_Bin', 'Changed_Credit_Limit_Bin', 'Num_of_Delayed_Payment_Bin', 'Credit_History_Age_Bin', 'Month_Num']

Amostra treino (head 20):
      Age Occupation  Annual_Income  Monthly_Inhand_Salary  Num_Bank_Accounts  \
0    23.0  Sc

## Codificação Final e Alinhamento dos Dados Curados

Esta etapa aplica a codificação final de variáveis categóricas usando `get_dummies()`, garantindo que todas as variáveis sejam numéricas e adequadas para modelagem supervisionada. Em seguida, alinha os conjuntos de treino e teste para que possuam exatamente as mesmas colunas, evitando inconsistências futuras. O resultado esperado é o salvamento dos arquivos finais `train_curated.csv` e `test_curated.csv` em `data/curated/`, prontos para rastreamento e versionamento com DVC.


In [ ]:
# 🔧 ETAPA: Codificação Final, Alinhamento e Versionamento dos Dados Curados

"""
Executa:
1) Carrega datasets feature engineered.
2) Identifica colunas não numéricas remanescentes.
3) Aplica get_dummies com drop_first para evitar multicolinearidade.
4) Alinha treino e teste.
5) Salva camada 'Curated' final.
6) Versiona com DVC.
Fluxo 100% rastreável, com tqdm e auditoria head(20).
"""

import os
import pandas as pd
from tqdm import tqdm
import subprocess

# 1️⃣ Validar CWD
print("Current Working Directory:", os.getcwd())

# 2️⃣ Paths
TRAIN_PATH = 'data/curated/train_feature_engineered.csv'
TEST_PATH = 'data/curated/test_feature_engineered.csv'
CURATED_TRAIN_PATH = 'data/curated/train_curated.csv'
CURATED_TEST_PATH = 'data/curated/test_curated.csv'

# 3️⃣ Carregar datasets
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

# 4️⃣ Identificar colunas não numéricas
train_non_numeric = train_df.select_dtypes(include=['object']).columns.tolist()
test_non_numeric = test_df.select_dtypes(include=['object']).columns.tolist()
print("\nColunas não numéricas (treino):", train_non_numeric)
print("Colunas não numéricas (teste):", test_non_numeric)

# 5️⃣ Codificação com tqdm
print("\nAplicando get_dummies...")
train_encoded = pd.get_dummies(train_df, drop_first=True)
test_encoded = pd.get_dummies(test_df, drop_first=True)

# 6️⃣ Alinhar colunas
print("\nAlinhando treino e teste...")
with tqdm(total=1, desc="Alinhamento") as pbar:
    train_aligned, test_aligned = train_encoded.align(test_encoded, join='left', axis=1, fill_value=0)
    pbar.update(1)

# 7️⃣ Prints de auditoria
print("\nTreino head(20):")
print(train_aligned.head(20))
print("\nTeste head(20):")
print(test_aligned.head(20))

# 8️⃣ Salvar versão final 'Curated'
train_aligned.to_csv(CURATED_TRAIN_PATH, index=False)
test_aligned.to_csv(CURATED_TEST_PATH, index=False)
print(f"\nArquivos salvos: {CURATED_TRAIN_PATH} | {CURATED_TEST_PATH}")

# 9️⃣ Versionamento com DVC
print("\nVersionando com DVC...")
with tqdm(total=3, desc="DVC Pipeline") as pbar:
    subprocess.run(['dvc', 'add', CURATED_TRAIN_PATH])
    pbar.update(1)
    subprocess.run(['dvc', 'add', CURATED_TEST_PATH])
    pbar.update(1)
    subprocess.run(['git', 'add', CURATED_TRAIN_PATH + '.dvc', CURATED_TEST_PATH + '.dvc'])
    subprocess.run(['git', 'commit', '-m', 'Add final curated train and test datasets'])
    subprocess.run(['dvc', 'push'])
    pbar.update(1)

print("\nCamada 'Curated' final criada, alinhada, versionada e enviada ao backend remoto.")


Current Working Directory: /workspace

Colunas não numéricas (treino): ['Occupation', 'Type_of_Loan', 'Credit_Mix', 'Payment_of_Min_Amount', 'Payment_Behaviour', 'Credit_Score']
Colunas não numéricas (teste): ['Type_of_Loan', 'Payment_of_Min_Amount']

Aplicando get_dummies...

Alinhando treino e teste...


Alinhamento: 100%|██████████| 1/1 [00:01<00:00,  1.07s/it]



Treino head(20):
      Age  Annual_Income  Monthly_Inhand_Salary  Num_Bank_Accounts  \
0    23.0       19114.12            1824.843333                  3   
1    23.0       19114.12                    NaN                  3   
2  -500.0       19114.12                    NaN                  3   
3    23.0       19114.12                    NaN                  3   
4    23.0       19114.12            1824.843333                  3   
5    23.0       19114.12                    NaN                  3   
6    23.0       19114.12            1824.843333                  3   
7    23.0       19114.12            1824.843333                  3   
8    33.0       34847.84            3037.986667                  2   
9    28.0       34847.84            3037.986667                  2   
10   28.0       37550.74            3037.986667                  2   
11   28.0       34847.84                    NaN                  2   
12   28.0       34847.84            3037.986667                  2   
13

DVC Pipeline:  67%|██████▋   | 2/3 [00:20<00:09,  9.28s/it]

[main 0c38ca0] Add final curated train and test datasets
 2 files changed, 10 insertions(+)
 create mode 100644 data/curated/test_curated.csv.dvc
 create mode 100644 data/curated/train_curated.csv.dvc


## Auditoria Final dos Arquivos Curated

Esta etapa confirma o tamanho real dos arquivos `train_curated.csv` e `test_curated.csv` salvos na camada `Curated`. Além disso, apresenta informações detalhadas de linhas, colunas e tipos de dados para verificar consistência antes de executar qualquer `dvc push` novamente. O objetivo é garantir rastreabilidade total do pipeline e evitar duplicidade de operações no backend.


In [1]:
## ETAPA: Auditoria Final — Tamanho e Estrutura dos Arquivos Curated

"""
Verifica:
1) Existência e tamanho em bytes/MB dos arquivos.
2) Dimensões (linhas x colunas).
3) Tipos de dados para garantir coerência.
"""

import os
import pandas as pd

# 1️⃣ Definir paths
CURATED_TRAIN_PATH = 'data/curated/train_curated.csv'
CURATED_TEST_PATH = 'data/curated/test_curated.csv'

# 2️⃣ Verificar tamanho dos arquivos
def file_size(path):
    size_bytes = os.path.getsize(path)
    size_mb = size_bytes / (1024 * 1024)
    return f"{size_mb:.2f} MB"

print(f"Tamanho train_curated.csv: {file_size(CURATED_TRAIN_PATH)}")
print(f"Tamanho test_curated.csv:  {file_size(CURATED_TEST_PATH)}")

# 3️⃣ Carregar e mostrar estrutura resumida
train_df = pd.read_csv(CURATED_TRAIN_PATH, nrows=100)  # carrega parcial para info
test_df = pd.read_csv(CURATED_TEST_PATH, nrows=100)

print("\nTreino — Amostra:")
print(f"Linhas (estimado pelo arquivo): variável")
print(f"Colunas: {len(train_df.columns)}")
print(train_df.info())

print("\nTeste — Amostra:")
print(f"Linhas (estimado pelo arquivo): variável")
print(f"Colunas: {len(test_df.columns)}")
print(test_df.info())


Tamanho train_curated.csv: 3610.34 MB
Tamanho test_curated.csv:  1805.16 MB

Treino — Amostra:
Linhas (estimado pelo arquivo): variável
Colunas: 6305
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Columns: 6305 entries, Age to Credit_Score_Standard
dtypes: bool(6287), float64(13), int64(5)
memory usage: 628.2 KB
None

Teste — Amostra:
Linhas (estimado pelo arquivo): variável
Colunas: 6305
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Columns: 6305 entries, Age to Credit_Score_Standard
dtypes: bool(6285), float64(13), int64(7)
memory usage: 629.5 KB
None


---
Refazendo Feature Engineering após estouro de Kernel por alta cardinalidade
---

---

---
PONTO DE PARTIDA
Reinício do Feature Engineering usando TRAIN_CLEAN_V1 e TEST_CLEAN_V1
Objetivo: reduzir cardinalidade de forma rastreável, evitando estouros de memória.
---


## ETAPA: INÍCIO DO FEATURE ENGINEERING — BASE V1

Começamos o Feature Engineering usando:
- `train_clean_v1.csv` como camada base (100k x 24)
- `test_clean_v1.csv` como camada base (50k x 23)

A versão anterior do curated gerou estouro de Kernel devido à cardinalidade excessiva.
Por isso, toda nova análise de cardinalidade, binning, encoding e seleção de features será refeita sobre a V1,
mantendo `Occupation` nominal até o momento certo de codificar.

PROTOCOLO V5.4 ativo: cada etapa terá validação de `df.shape` e `df.head(20)`.


In [18]:
#  ETAPA: CARREGAR BASES TRAIN_CLEAN_V1 E TEST_CLEAN_V1

"""
1️⃣ Carrega as versões limpas v1.
2️⃣ Confirma shapes.
3️⃣ Exibe amostra para verificar coerência.
"""

import pandas as pd

# Caminhos base
TRAIN_CLEAN_V1 = '../data/processed/train_clean_v1.csv'
TEST_CLEAN_V1  = '../data/processed/test_clean_v1.csv'

# 1️⃣ Carrega
df_train = pd.read_csv(TRAIN_CLEAN_V1)
df_test  = pd.read_csv(TEST_CLEAN_V1)

# 2️⃣ Confirma shapes
print(f"✅ TRAIN_CLEAN_V1 shape: {df_train.shape}")
print(f"✅ TEST_CLEAN_V1 shape: {df_test.shape}")

# 3️⃣ Amostras
print("\n🔍 Amostra TRAIN:")
print(df_train.head(20))

print("\n🔍 Amostra TEST:")
print(df_test.head(20))


✅ TRAIN_CLEAN_V1 shape: (100000, 24)
✅ TEST_CLEAN_V1 shape: (50000, 23)

🔍 Amostra TRAIN:
       Month   Age Occupation  Annual_Income  Monthly_Inhand_Salary  \
0    January  23.0  Scientist       19114.12            1824.843333   
1   February  23.0  Scientist       19114.12            3260.465000   
2      March  33.0  Scientist       19114.12            3260.465000   
3      April  23.0  Scientist       19114.12            3260.465000   
4        May  23.0  Scientist       19114.12            1824.843333   
5       June  23.0  Scientist       19114.12            3260.465000   
6       July  23.0  Scientist       19114.12            1824.843333   
7     August  23.0  Scientist       19114.12            1824.843333   
8    January  33.0      Other       34847.84            3037.986667   
9   February  28.0    Teacher       34847.84            3037.986667   
10     March  28.0    Teacher       35007.33            3037.986667   
11     April  28.0    Teacher       34847.84            30

/tmp/ipykernel_32837/1839673080.py:16: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv(TRAIN_CLEAN_V1)


## ETAPA: REANÁLISE DE CARDINALIDADE — BASE CLEAN V1

Esta etapa recalcula a cardinalidade real das colunas usando `TRAIN_CLEAN_V1` e `TEST_CLEAN_V1`,
para substituir o relatório antigo, que era baseado em uma versão não curada.

O novo `cardinality_report_v1.csv` permitirá:
- Identificar variáveis com alta cardinalidade real.
- Guiar binning, agrupamentos e transformações posteriores.
- Registrar decisões de forma rastreável no PROTOCOLO V5.4.


In [19]:
# ETAPA: CALCULAR NOVO RELATÓRIO DE CARDINALIDADE — V1

"""
1) Carrega TRAIN_CLEAN_V1 e TEST_CLEAN_V1.
2) Calcula cardinalidade (número de valores únicos) para cada coluna.
3) Gera DataFrame com Set (Train/Test), Column, UniqueValues.
4) Salva como cardinality_report_v1.csv.
"""

import pandas as pd

# Caminhos
TRAIN_CLEAN_V1 = '../data/processed/train_clean_v1.csv'
TEST_CLEAN_V1  = '../data/processed/test_clean_v1.csv'

df_train = pd.read_csv(TRAIN_CLEAN_V1, low_memory=False)
df_test  = pd.read_csv(TEST_CLEAN_V1, low_memory=False)

# Função para contar valores únicos
def get_cardinality(df, set_name):
    return pd.DataFrame({
        'Set': set_name,
        'Column': df.columns,
        'UniqueValues': [df[col].nunique(dropna=True) for col in df.columns]
    })

# Calcula para TRAIN e TEST
card_train = get_cardinality(df_train, 'Train')
card_test  = get_cardinality(df_test, 'Test')

# Combina
cardinality_report = pd.concat([card_train, card_test], ignore_index=True)

# Ordena para visualização
cardinality_report = cardinality_report.sort_values(by=['Set', 'UniqueValues'], ascending=[True, False])

# Salva
REPORT_PATH = '../data/curated/cardinality_report_v1.csv'
cardinality_report.to_csv(REPORT_PATH, index=False)

print("Relatório de cardinalidade gerado e salvo em:", REPORT_PATH)
print(cardinality_report)


Relatório de cardinalidade gerado e salvo em: ../data/curated/cardinality_report_v1.csv
      Set                    Column  UniqueValues
40   Test  Credit_Utilization_Ratio         50000
46   Test           Monthly_Balance         49433
44   Test   Amount_invested_monthly         45450
43   Test       Total_EMI_per_month         13144
27   Test             Annual_Income         12953
28   Test     Monthly_Inhand_Salary         12794
39   Test          Outstanding_Debt         12206
33   Test              Type_of_Loan          6260
36   Test      Changed_Credit_Limit          3921
31   Test             Interest_Rate           945
25   Test                       Age           890
30   Test           Num_Credit_Card           819
37   Test      Num_Credit_Inquiries           750
29   Test         Num_Bank_Accounts           539
41   Test        Credit_History_Age           399
35   Test    Num_of_Delayed_Payment           398
32   Test               Num_of_Loan           245
34   Test   

## ETAPA: ANÁLISE FINAL DE CARDINALIDADE CLEAN V1

- O novo relatório substitui a versão antiga que analisava a base não curada.
- Identificamos colunas com risco crítico de explosão de memória se forem codificadas sem binning prévio:
  - `Credit_Utilization_Ratio` (Train: 100k únicos, Test: 50k)
  - `Monthly_Balance`, `Amount_invested_monthly` e `Total_EMI_per_month` com dezenas de milhares de valores únicos.
  - `Type_of_Loan` com 6.260 combinações.
- Variáveis categóricas genuínas, como `Occupation` (16 classes), `Payment_Behaviour` (7) e `Credit_Mix` (4), podem ser mantidas para OHE ou Label Encoding controlado.
- Variável alvo `Credit_Score` no Train confirmada com 3 classes.


In [22]:
# ETAPA: INSPEÇÃO DIRETA DOS UNIQUES DE TYPE_OF_LOAN

"""
1) Carrega TRAIN_CLEAN_V1.
2) Mostra quantos valores únicos reais existem.
3) Exibe amostra dos valores únicos.
"""

import pandas as pd

TRAIN_CLEAN_V1 = '../data/processed/train_clean_v1.csv'
df_train = pd.read_csv(TRAIN_CLEAN_V1, low_memory=False)

# Verifica cardinalidade
unique_values = df_train['Type_of_Loan'].dropna().unique()
print(f"Total de combinações únicas: {len(unique_values)}\n")

# Exibe amostra organizada
for idx, val in enumerate(sorted(unique_values)[:50]):
    print(f"{idx+1}: {val}")

print("\n...Mostrando os 50 primeiros. Ajuste o slice para mais, se quiser.")


Total de combinações únicas: 6260

1: Auto Loan
2: Auto Loan, Auto Loan, Auto Loan, Auto Loan, Credit-Builder Loan, Credit-Builder Loan, Mortgage Loan, and Personal Loan
3: Auto Loan, Auto Loan, Auto Loan, Auto Loan, Student Loan, and Student Loan
4: Auto Loan, Auto Loan, Auto Loan, Credit-Builder Loan, Payday Loan, Not Specified, Payday Loan, Student Loan, and Debt Consolidation Loan
5: Auto Loan, Auto Loan, Auto Loan, Not Specified, Debt Consolidation Loan, and Credit-Builder Loan
6: Auto Loan, Auto Loan, Auto Loan, Not Specified, and Home Equity Loan
7: Auto Loan, Auto Loan, Auto Loan, Personal Loan, Student Loan, and Not Specified
8: Auto Loan, Auto Loan, Auto Loan, and Debt Consolidation Loan
9: Auto Loan, Auto Loan, Auto Loan, and Home Equity Loan
10: Auto Loan, Auto Loan, Auto Loan, and Not Specified
11: Auto Loan, Auto Loan, Auto Loan, and Personal Loan
12: Auto Loan, Auto Loan, Auto Loan, and Student Loan
13: Auto Loan, Auto Loan, Credit-Builder Loan, Not Specified, Not Specif

## ETAPA: DEFINIÇÃO DA REGRA PARA SIMPLIFICAR TYPE_OF_LOAN

Para evitar a explosão de cardinalidade causada pelas múltiplas combinações de produtos financeiros na coluna `Type_of_Loan`, definimos uma política de **hierarquia de risco de crédito**.

A ideia é **reduzir qualquer combinação de tipos de empréstimo para uma única classe**, escolhendo o tipo mais restritivo (maior risco) presente na carteira do cliente.

Esta decisão é baseada em uma interpretação de mercado onde produtos como **Payday Loan** são mais arriscados, enquanto produtos como **Mortgage Loan** ou **Home Equity Loan** são considerados de menor risco. Assim, o cliente é classificado pelo tipo de empréstimo que mais compromete seu perfil de crédito.

A ordem de risco adotada é:
1. Payday Loan
2. Credit-Builder Loan
3. Debt Consolidation Loan
4. Personal Loan
5. Student Loan
6. Auto Loan
7. Home Equity Loan
8. Mortgage Loan
9. Not Specified (casos sem classificação clara)

Dessa forma, cada linha da base passa a ter uma coluna `Type_of_Loan_Category` com a classe única mais restritiva. Esta variável substitui a combinação original e permite codificação controlada nos próximos passos de engenharia de atributos.


In [24]:
# ETAPA: CRIAR CATEGORIA HIERÁRQUICA PARA TYPE_OF_LOAN

"""
1) Define hierarquia de risco.
2) Para cada registro, identifica todos os tipos de empréstimo.
3) Seleciona o tipo de maior risco com base na hierarquia.
4) Cria coluna Type_of_Loan_Category.
"""

import pandas as pd

# Caminhos de trabalho
TRAIN_CLEAN_V1 = '../data/processed/train_clean_v1.csv'
TEST_CLEAN_V1  = '../data/processed/test_clean_v1.csv'

df_train = pd.read_csv(TRAIN_CLEAN_V1, low_memory=False)
df_test  = pd.read_csv(TEST_CLEAN_V1, low_memory=False)

# 1) Hierarquia: menor índice = maior risco
loan_risk_rank = [
    'Payday Loan',
    'Credit-Builder Loan',
    'Debt Consolidation Loan',
    'Personal Loan',
    'Student Loan',
    'Auto Loan',
    'Home Equity Loan',
    'Mortgage Loan',
    'Not Specified'
]

def classify_highest_risk(loan_string):
    if pd.isna(loan_string):
        return 'Not Specified'
    # Substitui ' and ' por ', ' para padronizar
    loan_string = loan_string.replace(' and ', ', ')
    tokens = [s.strip() for s in loan_string.split(',') if s.strip()]
    if not tokens:
        return 'Not Specified'
    # Identifica tipo de maior risco
    tokens = [t if t in loan_risk_rank else 'Not Specified' for t in tokens]
    ranked_tokens = sorted(tokens, key=lambda x: loan_risk_rank.index(x))
    return ranked_tokens[0]

# 2) Aplica ao TRAIN
df_train['Type_of_Loan_Category'] = df_train['Type_of_Loan'].apply(classify_highest_risk)

# 3) Aplica ao TEST
df_test['Type_of_Loan_Category'] = df_test['Type_of_Loan'].apply(classify_highest_risk)

# 4) Verifica distribuição
print("\nDistribuição TRAIN:")
print(df_train['Type_of_Loan_Category'].value_counts())

print("\nDistribuição TEST:")
print(df_test['Type_of_Loan_Category'].value_counts())

# 5) Opcional: salva snapshot
df_train.to_csv('../data/processed/train_clean_v1_with_loan_category.csv', index=False)
df_test.to_csv('../data/processed/test_clean_v1_with_loan_category.csv', index=False)
print("\nArquivos atualizados com Type_of_Loan_Category salvos.")



Distribuição TRAIN:
Type_of_Loan_Category
Payday Loan                31944
Credit-Builder Loan        19984
Not Specified              12968
Debt Consolidation Loan    12480
Personal Loan               8416
Student Loan                5856
Auto Loan                   3616
Home Equity Loan            2760
Mortgage Loan               1976
Name: count, dtype: int64

Distribuição TEST:
Type_of_Loan_Category
Payday Loan                15972
Credit-Builder Loan         9992
Not Specified               6484
Debt Consolidation Loan     6240
Personal Loan               4208
Student Loan                2928
Auto Loan                   1808
Home Equity Loan            1380
Mortgage Loan                988
Name: count, dtype: int64

Arquivos atualizados com Type_of_Loan_Category salvos.


## ETAPA: FINALIZAÇÃO DA HIERARQUIA TYPE_OF_LOAN

A coluna original `Type_of_Loan` foi transformada na variável `Type_of_Loan_Category`
aplicando uma regra de negócio de hierarquia de risco:
- Para cada combinação, atribuiu-se a classe do tipo de empréstimo mais restritivo.
- O total soma 100% dos registros em TRAIN e TEST, sem sobreposição.
- Esta nova variável será usada como insumo para codificação controlada (Label Encoding ou OHE) na próxima etapa.


## ETAPA: ONE-HOT ENCODING PARA TYPE_OF_LOAN_CATEGORY

Com a coluna `Type_of_Loan_Category` criada e salva na camada `staged`,
aplicaremos agora o One-Hot Encoding (OHE) de forma controlada.

A codificação será aplicada em `train` e `test` dentro da `staged`,
gerando novos arquivos `train_staged_v1_with_loan_ohe.csv` e `test_staged_v1_with_loan_ohe.csv`.

Cada categoria passa a ter uma coluna binária (0 ou 1) para facilitar a entrada no modelo.


In [3]:
# ETAPA: APLICAR OHE EM TYPE_OF_LOAN_CATEGORY E SALVAR EM STAGED

"""
1) Carrega os arquivos staged.
2) Aplica pd.get_dummies em Type_of_Loan_Category.
3) Salva arquivos atualizados na pasta staged.
"""

import pandas as pd

# Caminhos staged
STAGED_TRAIN_PATH = '../data/staged/train_clean_v1_with_loan_category.csv'
STAGED_TEST_PATH  = '../data/staged/test_clean_v1_with_loan_category.csv'

df_train = pd.read_csv(STAGED_TRAIN_PATH)
df_test  = pd.read_csv(STAGED_TEST_PATH)

# Confirma coluna
print("\nColunas TRAIN antes do OHE:", df_train.columns.tolist())

# One-Hot Encoding
df_train_ohe = pd.get_dummies(df_train, columns=['Type_of_Loan_Category'])
df_test_ohe  = pd.get_dummies(df_test,  columns=['Type_of_Loan_Category'])

# Confirma novas colunas
print("\nColunas TRAIN depois do OHE:", df_train_ohe.columns.tolist())

# Paths de saída
TRAIN_STAGED_OHE_PATH = '../data/staged/train_staged_v1_with_loan_ohe.csv'
TEST_STAGED_OHE_PATH  = '../data/staged/test_staged_v1_with_loan_ohe.csv'

# Salva
df_train_ohe.to_csv(TRAIN_STAGED_OHE_PATH, index=False)
df_test_ohe.to_csv(TEST_STAGED_OHE_PATH, index=False)

print("\nArquivos com OHE salvos em staged:")
print(f"- {TRAIN_STAGED_OHE_PATH}")
print(f"- {TEST_STAGED_OHE_PATH}")


/tmp/ipykernel_104189/3542176793.py:15: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv(STAGED_TRAIN_PATH)



Colunas TRAIN antes do OHE: ['Month', 'Age', 'Occupation', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Type_of_Loan', 'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit', 'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt', 'Credit_Utilization_Ratio', 'Credit_History_Age', 'Payment_of_Min_Amount', 'Total_EMI_per_month', 'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance', 'Credit_Score', 'Type_of_Loan_Category']

Colunas TRAIN depois do OHE: ['Month', 'Age', 'Occupation', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Type_of_Loan', 'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit', 'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt', 'Credit_Utilization_Ratio', 'Credit_History_Age', 'Payment_of_Min_Amount', 'Total_EMI_per_month', 'Amount_invested_monthly', 'Payment_Behaviour', 'Mont

## ETAPA: INSPEÇÃO E AUDITORIA DE TIPOS NUMÉRICOS E MISTOS — STAGED V1

Neste ponto do pipeline, já consolidamos:
- Os dados `train` e `test` na camada `staged` com `Type_of_Loan_Category` classificado e codificado via OHE.
- As principais variáveis categóricas.

Agora o foco volta para as **variáveis numéricas e de tipos mistos**, que podem conter:
- Placeholders não convertidos (ex: `_` ou strings residuais).
- Valores impossíveis (ex: negativos em dívida, saldo ou percentual de utilização de crédito).
- Tipos `object` que deveriam ser `float64`.

Esta etapa faz uma auditoria de `dtypes` e mostra amostras para decidir:
- Se é necessário coerção de tipos (`pd.to_numeric`).
- Se aplicamos binning ou tratamento de outliers.
- Quais colunas precisam de ajustes adicionais antes de seguir para o `curated`.

Após esta verificação, cada coluna será corrigida ou agrupada seguindo o PROTOCOLO V5.4.


In [4]:
# ETAPA: INSPEÇÃO DOS TIPOS NUMÉRICOS E MISTOS — STAGED V1

"""
1) Carrega os arquivos staged com OHE.
2) Mostra df.dtypes.
3) Exibe amostra de colunas numéricas críticas.
"""

import pandas as pd

TRAIN_STAGED_OHE_PATH = '../data/staged/train_staged_v1_with_loan_ohe.csv'
TEST_STAGED_OHE_PATH  = '../data/staged/test_staged_v1_with_loan_ohe.csv'

df_train = pd.read_csv(TRAIN_STAGED_OHE_PATH, low_memory=False)
df_test  = pd.read_csv(TEST_STAGED_OHE_PATH, low_memory=False)

print("\nTipos de dados TRAIN:")
print(df_train.dtypes)

cols_to_check = [
    'Credit_Utilization_Ratio',
    'Outstanding_Debt',
    'Monthly_Balance',
    'Amount_invested_monthly'
]

print("\nAmostra TRAIN:")
print(df_train[cols_to_check].head(20))



Tipos de dados TRAIN:
Month                                             object
Age                                              float64
Occupation                                        object
Annual_Income                                    float64
Monthly_Inhand_Salary                            float64
Num_Bank_Accounts                                  int64
Num_Credit_Card                                    int64
Interest_Rate                                      int64
Num_of_Loan                                      float64
Type_of_Loan                                      object
Delay_from_due_date                                int64
Num_of_Delayed_Payment                           float64
Changed_Credit_Limit                             float64
Num_Credit_Inquiries                             float64
Credit_Mix                                        object
Outstanding_Debt                                 float64
Credit_Utilization_Ratio                         float64
Credit_H

## ETAPA: COERÇÃO E IMPUTAÇÃO AGRUPADA POR OCCUPATION

Conforme definido no início do pipeline, o tratamento de variáveis numéricas
com tipos mistos deve seguir os seguintes princípios:

1) Garantir que a coluna `Occupation` esteja completa:
   - Substituir placeholders ('Unknown', '_', 'Other', '_______', '!@9#%8', '#F%$D@*&8') por `Other`.
   - Imputar `NaN` com a moda global de `Occupation`.

2) Para cada coluna numérica correlacionada ao perfil ocupacional:
   - Substituir placeholders ('_', '__10000__', 'Other') por `NaN`.
   - Coerção segura para `float64`.
   - Imputar `NaN` com mediana agrupada por `Occupation`.
   - Se `Occupation` não existir para o registro, fallback para mediana global.

3) Colunas não relacionadas a `Occupation` devem usar imputação global.

Os arquivos resultantes serão salvos na camada `staged` com rastreio incremental.


In [5]:
# ETAPA: COERÇÃO E IMPUTAÇÃO AGRUPADA POR OCCUPATION — STAGED V1

"""
1) Padroniza a coluna Occupation.
2) Para colunas numéricas correlacionadas, substitui placeholders, coerção para float, imputação por groupby(Occupation).
3) Salva versões coeridas na pasta staged.
"""

import pandas as pd

# Caminhos staged
TRAIN_STAGED_OHE_PATH = '../data/staged/train_staged_v1_with_loan_ohe.csv'
TEST_STAGED_OHE_PATH  = '../data/staged/test_staged_v1_with_loan_ohe.csv'

df_train = pd.read_csv(TRAIN_STAGED_OHE_PATH, low_memory=False)
df_test  = pd.read_csv(TEST_STAGED_OHE_PATH, low_memory=False)

# 1) Padronizar Occupation
placeholders_occ = ['Unknown', '_', 'Other', '_______', '!@9#%8', '#F%$D@*&8']
for df in [df_train, df_test]:
    if 'Occupation' in df.columns:
        df['Occupation'] = df['Occupation'].replace(placeholders_occ, 'Other')
        df['Occupation'] = df['Occupation'].fillna(df['Occupation'].mode()[0])

# 2) Variáveis correlacionadas
cols_grouped = ['Amount_invested_monthly', 'Monthly_Balance']

placeholders_num = ['_', '__10000__', 'Other']

for df in [df_train, df_test]:
    for col in cols_grouped:
        if col in df.columns:
            df[col] = df[col].replace(placeholders_num, pd.NA)
            df[col] = pd.to_numeric(df[col], errors='coerce')
            df[col] = df.groupby('Occupation')[col].transform(
                lambda x: x.fillna(x.median())
            )
            # Fallback se ainda restar NaN
            df[col] = df[col].fillna(df[col].median())

# Verificação
print("\nTipos TRAIN pós-coerção:")
print(df_train[cols_grouped].dtypes)
print("\nAmostra TRAIN pós-coerção:")
print(df_train[cols_grouped].head(20))

# Salvar na staged
TRAIN_STAGED_GROUPED_PATH = '../data/staged/train_staged_v1_num_coerced_by_occupation.csv'
TEST_STAGED_GROUPED_PATH  = '../data/staged/test_staged_v1_num_coerced_by_occupation.csv'

df_train.to_csv(TRAIN_STAGED_GROUPED_PATH, index=False)
df_test.to_csv(TEST_STAGED_GROUPED_PATH, index=False)

print("\nArquivos salvos com coerção agrupada por Occupation na pasta staged.")



Tipos TRAIN pós-coerção:
Amount_invested_monthly    float64
Monthly_Balance            float64
dtype: object

Amostra TRAIN pós-coerção:
    Amount_invested_monthly  Monthly_Balance
0                 80.415295       312.494089
1                118.280222       284.629162
2                 81.699521       331.209863
3                199.458074       223.451310
4                 41.420153       341.489231
5                 62.430172       340.479212
6                178.344067       244.565317
7                 24.785217       358.124168
8                104.291825       470.690627
9                 40.391238       484.591214
10                58.515976       466.466476
11                99.306228       465.676224
12               130.115420       444.867032
13                43.477190       481.505262
14                70.101774       464.880678
15               218.904344       356.078109
16               168.413703      1043.315978
17               232.860384       998.869297
18     

## ETAPA: BINNING INTERPRETÁVEL PARA VARIÁVEIS NUMÉRICAS

Nesta etapa, aplicaremos binning em todas as variáveis numéricas com cardinalidade elevada
ou onde faixas discretas agregam mais sentido do que normalização bruta.

Critérios:
- Diagnóstico de faixa real com `describe()`.
- Bins manuais ajustados para intervalos financeiros ou de risco coerentes.
- Labels claros para rastreabilidade.
- Aplicação em `TRAIN` e `TEST` na camada `staged`.

PROTOCOLO V5.4: Cada faixa é documentada, validada e salva incrementalmente.


In [11]:
# ETAPA: BINNING ROBUSTO FINAL — MONOTONICIDADE GARANTIDA

import pandas as pd

TRAIN_PATH = '../data/staged/train_staged_v1_num_coerced_by_occupation.csv'
TEST_PATH  = '../data/staged/test_staged_v1_num_coerced_by_occupation.csv'

df_train = pd.read_csv(TRAIN_PATH)
df_test  = pd.read_csv(TEST_PATH)

# ==============================
# Função para variáveis financeiras
def generate_bins_financial(var, penultimate, default_bins, labels):
    max_value = df_train[var].max()
    if max_value <= penultimate:
        bins = [default_bins[0]] + default_bins[1:-2] + [max_value + 1]
        labels_adj = labels[:len(bins)-1]
    else:
        bins = default_bins[:-1] + [max_value + 1]
        labels_adj = labels
    print(f"\n{var} | max={max_value:.2f} | bins={bins} | labels={labels_adj}")
    return bins, labels_adj

# Função para variáveis percentuais
def generate_bins_percentual(var):
    max_value = df_train[var].max()
    if max_value <= 100:
        bins = [0, 30, 50, 75, 100]
        labels = ['Low', 'Moderate', 'High', 'Very High']
    else:
        bins = [0, 30, 50, 75, 100, max_value + 1]
        labels = ['Low', 'Moderate', 'High', 'Very High', 'Extreme']
    print(f"\n{var} | max={max_value:.2f} | bins={bins} | labels={labels}")
    return bins, labels

# ==============================
# Diagnóstico e geração
print("\nAmount_invested_monthly describe:")
print(df_train['Amount_invested_monthly'].describe())
bins_aim, labels_aim = generate_bins_financial(
    'Amount_invested_monthly',
    2000,
    [0, 500, 1000, 2000, 'max'],
    ['Low', 'Moderate', 'High', 'Very High']
)

print("\nMonthly_Balance describe:")
print(df_train['Monthly_Balance'].describe())
bins_mb, labels_mb = generate_bins_financial(
    'Monthly_Balance',
    2000,
    [0, 500, 1000, 2000, 'max'],
    ['Low', 'Moderate', 'High', 'Very High']
)

print("\nOutstanding_Debt describe:")
print(df_train['Outstanding_Debt'].describe())
bins_od, labels_od = generate_bins_financial(
    'Outstanding_Debt',
    2000,
    [0, 500, 1000, 2000, 'max'],
    ['Low', 'Moderate', 'High', 'Very High']
)

print("\nCredit_Utilization_Ratio describe:")
print(df_train['Credit_Utilization_Ratio'].describe())
bins_cur, labels_cur = generate_bins_percentual('Credit_Utilization_Ratio')

# ==============================
# Aplicar pd.cut() robusto
for df in [df_train, df_test]:
    df['Amount_invested_monthly_Binned'] = pd.cut(
        df['Amount_invested_monthly'],
        bins=bins_aim,
        labels=labels_aim,
        include_lowest=True
    )
    df['Monthly_Balance_Binned'] = pd.cut(
        df['Monthly_Balance'],
        bins=bins_mb,
        labels=labels_mb,
        include_lowest=True
    )
    df['Outstanding_Debt_Binned'] = pd.cut(
        df['Outstanding_Debt'],
        bins=bins_od,
        labels=labels_od,
        include_lowest=True
    )
    df['Credit_Utilization_Ratio_Binned'] = pd.cut(
        df['Credit_Utilization_Ratio'],
        bins=bins_cur,
        labels=labels_cur,
        include_lowest=True
    )

# ==============================
# Validar distribuições
print("\nDistribuição Amount_invested_monthly_Binned:")
print(df_train['Amount_invested_monthly_Binned'].value_counts())

print("\nDistribuição Monthly_Balance_Binned:")
print(df_train['Monthly_Balance_Binned'].value_counts())

print("\nDistribuição Outstanding_Debt_Binned:")
print(df_train['Outstanding_Debt_Binned'].value_counts())

print("\nDistribuição Credit_Utilization_Ratio_Binned:")
print(df_train['Credit_Utilization_Ratio_Binned'].value_counts())

# ==============================
# Salvar em staged
TRAIN_PATH_BINS = '../data/staged/train_staged_v1_binned.csv'
TEST_PATH_BINS  = '../data/staged/test_staged_v1_binned.csv'

df_train.to_csv(TRAIN_PATH_BINS, index=False)
df_test.to_csv(TEST_PATH_BINS, index=False)

print("\nArquivos com binning interpretável e robusto salvos em staged:")
print(f"- {TRAIN_PATH_BINS}")
print(f"- {TEST_PATH_BINS}")



Amount_invested_monthly describe:
count    100000.000000
mean        189.695054
std         191.527989
min           0.000000
25%          77.017414
50%         129.035357
75%         220.039055
max        1977.326102
Name: Amount_invested_monthly, dtype: float64

Amount_invested_monthly | max=1977.33 | bins=[0, 500, 1000, np.float64(1978.326102249349)] | labels=['Low', 'Moderate', 'High']

Monthly_Balance describe:
count    100000.000000
mean        401.757116
std         212.749898
min           0.007760
25%         270.913865
50%         336.762318
75%         467.670597
max        1602.040519
Name: Monthly_Balance, dtype: float64

Monthly_Balance | max=1602.04 | bins=[0, 500, 1000, np.float64(1603.0405189622518)] | labels=['Low', 'Moderate', 'High']

Outstanding_Debt describe:
count    100000.000000
mean       1423.862010
std        1149.508317
min           0.230000
25%         571.650000
50%        1167.200000
75%        1933.430000
max        4998.070000
Name: Outstanding_Debt,

## ETAPA: VALIDAÇÃO CRUZADA DOS BINS ENTRE TRAIN E TEST

Para garantir que o pipeline de modelagem funcione sem inconsistências,
vamos validar se as categorias geradas por `pd.cut()` estão coerentes
em `train` e `test`.

- Se uma faixa estiver ausente em `test`, será ajustada via `CategoricalDtype`.
- Isto previne erros de forma (shape mismatch) ao aplicar One-Hot Encoding.

PROTOCOLO V5.4: Cada verificação deve ser registrada com `value_counts()`.


In [12]:
# ETAPA: VALIDAÇÃO CRUZADA DOS BINS ENTRE TRAIN E TEST

import pandas as pd

TRAIN_PATH_BINS = '../data/staged/train_staged_v1_binned.csv'
TEST_PATH_BINS  = '../data/staged/test_staged_v1_binned.csv'

df_train = pd.read_csv(TRAIN_PATH_BINS)
df_test  = pd.read_csv(TEST_PATH_BINS)

# Variáveis binned
binned_cols = [
    'Amount_invested_monthly_Binned',
    'Monthly_Balance_Binned',
    'Outstanding_Debt_Binned',
    'Credit_Utilization_Ratio_Binned'
]

for col in binned_cols:
    print(f"\n=== {col} ===")
    print("\nTRAIN:")
    print(df_train[col].value_counts(dropna=False))
    print("\nTEST:")
    print(df_test[col].value_counts(dropna=False))



=== Amount_invested_monthly_Binned ===

TRAIN:
Amount_invested_monthly_Binned
Low         92841
Moderate     6242
High          917
Name: count, dtype: int64

TEST:
Amount_invested_monthly_Binned
Low         46454
Moderate     3146
High          400
Name: count, dtype: int64

=== Monthly_Balance_Binned ===

TRAIN:
Monthly_Balance_Binned
Low         78319
Moderate    19297
High         2384
Name: count, dtype: int64

TEST:
Monthly_Balance_Binned
Low         39106
Moderate     9712
High         1181
NaN             1
Name: count, dtype: int64

=== Outstanding_Debt_Binned ===

TRAIN:
Outstanding_Debt_Binned
High         33553
Very High    23949
Low          21756
Moderate     20742
Name: count, dtype: int64

TEST:
Outstanding_Debt_Binned
High         16753
Very High    11961
Low          10907
Moderate     10379
Name: count, dtype: int64

=== Credit_Utilization_Ratio_Binned ===

TRAIN:
Credit_Utilization_Ratio_Binned
Moderate    63637
Low         36362
High            1
Name: count, dtyp

## ETAPA: CODIFICAÇÃO FINAL DOS BINS INTERPRETÁVEIS

Após validar que as categorias dos bins são coerentes entre `TRAIN` e `TEST`,
aplicaremos a codificação final com:

- `CategoricalDtype` para garantir que as classes existam mesmo que uma não apareça.
- One-Hot Encoding (`pd.get_dummies()`) idêntico em `TRAIN` e `TEST`.
- Resolução de `NaN` residual em `Monthly_Balance_Binned` para evitar categorias ausentes.

Os arquivos resultantes serão salvos na camada `staged` como checkpoint
antes de seguir para o `curated`.


In [13]:
# ETAPA: CODIFICAÇÃO FINAL DOS BINS — OHE ROBUSTO

import pandas as pd
from pandas.api.types import CategoricalDtype

TRAIN_PATH_BINS = '../data/staged/train_staged_v1_binned.csv'
TEST_PATH_BINS  = '../data/staged/test_staged_v1_binned.csv'

df_train = pd.read_csv(TRAIN_PATH_BINS)
df_test  = pd.read_csv(TEST_PATH_BINS)

# Variáveis binned
binned_cols = [
    'Amount_invested_monthly_Binned',
    'Monthly_Balance_Binned',
    'Outstanding_Debt_Binned',
    'Credit_Utilization_Ratio_Binned'
]

# Categorias únicas por variável — baseadas na união TRAIN+TEST
categories_map = {
    'Amount_invested_monthly_Binned': ['Low', 'Moderate', 'High'],
    'Monthly_Balance_Binned': ['Low', 'Moderate', 'High'],
    'Outstanding_Debt_Binned': ['Low', 'Moderate', 'High', 'Very High'],
    'Credit_Utilization_Ratio_Binned': ['Low', 'Moderate', 'High']
}

# 1️⃣ Se houver NaN, imputar com moda para manter coerência
df_test['Monthly_Balance_Binned'] = df_test['Monthly_Balance_Binned'].fillna(
    df_train['Monthly_Balance_Binned'].mode()[0]
)

# 2️⃣ Fixar dtype categórico para cada coluna
for col, cats in categories_map.items():
    dtype = CategoricalDtype(categories=cats, ordered=True)
    df_train[col] = df_train[col].astype(dtype)
    df_test[col] = df_test[col].astype(dtype)

# 3️⃣ Aplicar One-Hot Encoding robusto
df_train_encoded = pd.get_dummies(df_train, columns=binned_cols, prefix=binned_cols)
df_test_encoded  = pd.get_dummies(df_test, columns=binned_cols, prefix=binned_cols)

# 4️⃣ Verificar colunas finais
print("\nColunas TRAIN OHE:", [c for c in df_train_encoded.columns if any(b in c for b in binned_cols)])
print("\nColunas TEST OHE:", [c for c in df_test_encoded.columns if any(b in c for b in binned_cols)])

# 5️⃣ Salvar na staged
TRAIN_PATH_ENCODED = '../data/staged/train_staged_v1_binned_encoded.csv'
TEST_PATH_ENCODED  = '../data/staged/test_staged_v1_binned_encoded.csv'

df_train_encoded.to_csv(TRAIN_PATH_ENCODED, index=False)
df_test_encoded.to_csv(TEST_PATH_ENCODED, index=False)

print("\nArquivos OHE salvos em staged:")
print(f"- {TRAIN_PATH_ENCODED}")
print(f"- {TEST_PATH_ENCODED}")



Colunas TRAIN OHE: ['Amount_invested_monthly_Binned_Low', 'Amount_invested_monthly_Binned_Moderate', 'Amount_invested_monthly_Binned_High', 'Monthly_Balance_Binned_Low', 'Monthly_Balance_Binned_Moderate', 'Monthly_Balance_Binned_High', 'Outstanding_Debt_Binned_Low', 'Outstanding_Debt_Binned_Moderate', 'Outstanding_Debt_Binned_High', 'Outstanding_Debt_Binned_Very High', 'Credit_Utilization_Ratio_Binned_Low', 'Credit_Utilization_Ratio_Binned_Moderate', 'Credit_Utilization_Ratio_Binned_High']

Colunas TEST OHE: ['Amount_invested_monthly_Binned_Low', 'Amount_invested_monthly_Binned_Moderate', 'Amount_invested_monthly_Binned_High', 'Monthly_Balance_Binned_Low', 'Monthly_Balance_Binned_Moderate', 'Monthly_Balance_Binned_High', 'Outstanding_Debt_Binned_Low', 'Outstanding_Debt_Binned_Moderate', 'Outstanding_Debt_Binned_High', 'Outstanding_Debt_Binned_Very High', 'Credit_Utilization_Ratio_Binned_Low', 'Credit_Utilization_Ratio_Binned_Moderate', 'Credit_Utilization_Ratio_Binned_High']

Arquivos

## ETAPA: CONSOLIDAÇÃO FINAL DO STAGED

Vamos consolidar:
- Todas as variáveis numéricas coeridas por `Occupation`.
- Todas as variáveis textuais limpas.
- As variáveis binned + OHE.

Faremos merge horizontal usando o índice como chave técnica.
A saída será `train_staged_v1_final.csv` e `test_staged_v1_final.csv`
prontos para validação de shape e versionamento com DVC.


In [14]:
# ETAPA: CONSOLIDAR NUMÉRICO, TEXTO E BINS EM UM ÚNICO STAGED FINAL

import pandas as pd

# 1) Caminhos
NUM_TRAIN_PATH = '../data/staged/train_staged_v1_num_coerced_by_occupation.csv'
NUM_TEST_PATH  = '../data/staged/test_staged_v1_num_coerced_by_occupation.csv'
BINS_TRAIN_PATH = '../data/staged/train_staged_v1_binned_encoded.csv'
BINS_TEST_PATH  = '../data/staged/test_staged_v1_binned_encoded.csv'

# 2) Carregar DataFrames
df_num_train = pd.read_csv(NUM_TRAIN_PATH)
df_num_test  = pd.read_csv(NUM_TEST_PATH)

df_bins_train = pd.read_csv(BINS_TRAIN_PATH)
df_bins_test  = pd.read_csv(BINS_TEST_PATH)

# 3) Alinhar índice e concatenar
df_num_train = df_num_train.reset_index(drop=True)
df_num_test  = df_num_test.reset_index(drop=True)
df_bins_train = df_bins_train.reset_index(drop=True)
df_bins_test  = df_bins_test.reset_index(drop=True)

df_train_final = pd.concat([df_num_train, df_bins_train], axis=1)
df_test_final  = pd.concat([df_num_test, df_bins_test], axis=1)

# 4) Remover colunas duplicadas (caso já existam do binning em coeridos)
df_train_final = df_train_final.loc[:, ~df_train_final.columns.duplicated()]
df_test_final  = df_test_final.loc[:, ~df_test_final.columns.duplicated()]

# 5) Verificar shape final
print("\nShape TRAIN final:", df_train_final.shape)
print("Shape TEST final:", df_test_final.shape)

# 6) Verificar se colunas estão alinhadas
cols_train = set(df_train_final.columns)
cols_test  = set(df_test_final.columns)

print("\nColunas que só existem no TRAIN:", cols_train - cols_test)
print("Colunas que só existem no TEST:", cols_test - cols_train)

# 7) Salvar versão final staged
FINAL_TRAIN_PATH = '../data/staged/train_staged_v1_final.csv'
FINAL_TEST_PATH  = '../data/staged/test_staged_v1_final.csv'

df_train_final.to_csv(FINAL_TRAIN_PATH, index=False)
df_test_final.to_csv(FINAL_TEST_PATH, index=False)

print("\nArquivos STAGED FINAL salvos:")
print(f"- {FINAL_TRAIN_PATH}")
print(f"- {FINAL_TEST_PATH}")



Shape TRAIN final: (100000, 46)
Shape TEST final: (50000, 45)

Colunas que só existem no TRAIN: {'Credit_Score'}
Colunas que só existem no TEST: set()

Arquivos STAGED FINAL salvos:
- ../data/staged/train_staged_v1_final.csv
- ../data/staged/test_staged_v1_final.csv


## ETAPA: CONSOLIDAÇÃO FINAL COMPLETA

- Merge concluído entre variáveis numéricas coeridas, textuais limpas, binning interpretável e OHE.
- `Credit_Score` presente apenas no `TRAIN`, mantendo separação entre features e target.
- `TRAIN` e `TEST` alinhados para shape e colunas de entrada.
- Arquivos salvos na camada `staged` como:
  - `train_staged_v1_final.csv`
  - `test_staged_v1_final.csv`


## ETAPA: VERSIONAMENTO COM DVC — PACOTE FINAL DA CAMADA STAGED

Conforme boas práticas:
- Vamos versionar os dois arquivos finais `staged`:
  - `train_staged_v1_final.csv`
  - `test_staged_v1_final.csv`
- Assim mantemos rastreabilidade do checkpoint **antes de promover para o `curated`**.
- O versionamento DVC + Git garante rollback limpo e rastreamento do snapshot.

PROTOCOLO V5.4: Nenhuma etapa não rastreada, tudo auditável.


In [21]:
# ETAPA: VERSIONAMENTO FINAL COM PYTHON PURO

import os
import subprocess

# Caminhos reais dos arquivos
train_file = '/workspace/data/staged/train_staged_v1_final.csv'
test_file  = '/workspace/data/staged/test_staged_v1_final.csv'

# Verifique se existem
print("\n✅ Verificando existência física dos arquivos:")
print("TRAIN:", os.path.exists(train_file))
print("TEST :", os.path.exists(test_file))

if not os.path.exists(train_file) or not os.path.exists(test_file):
    raise FileNotFoundError("🚫 Um dos arquivos não existe em /workspace/data/staged/")

# DVC ADD
print("\n✅ Executando dvc add ...")
subprocess.run(['dvc', 'add', train_file], check=True)
subprocess.run(['dvc', 'add', test_file], check=True)

# DVC STATUS
print("\n✅ Status do DVC:")
subprocess.run(['dvc', 'status'], check=True)

# GIT ADD (corrigido)
print("\n✅ Git add dos arquivos .dvc:")
git_add_cmd = [
    'git', 'add',
    f'{train_file}.dvc',
    f'{test_file}.dvc'
]

# Verifique se .gitignore existe
if os.path.exists('.gitignore'):
    print("Incluindo .gitignore no git add.")
    git_add_cmd.append('.gitignore')
else:
    print("⚠️ .gitignore não encontrado — ignorado do git add.")

subprocess.run(git_add_cmd, check=True)

# GIT COMMIT
print("\n✅ Git commit:")
subprocess.run(['git', 'commit', '-m',
                'Versiona STAGED V1 FINAL — coerção numérica, texto limpo, binning e OHE (/workspace/data/staged)'], check=True)

# DVC PUSH
print("\n✅ DVC push para remote:")
subprocess.run(['dvc', 'push'], check=True)

print("\n🚩 PROCESSO FINALIZADO — STAGED V1 FINAL VERSIONADO COM PYTHON PURO E CAMINHO CORRETO.")



✅ Verificando existência física dos arquivos:
TRAIN: True
TEST : True

✅ Executando dvc add ...


⠋ Checking graph
⠋ Checking graph



✅ Status do DVC:
Data and pipelines are up to date.

✅ Git add dos arquivos .dvc:
⚠️ .gitignore não encontrado — ignorado do git add.

✅ Git commit:
[main 115cfa0] Versiona STAGED V1 FINAL — coerção numérica, texto limpo, binning e OHE (/workspace/data/staged)
 2 files changed, 10 insertions(+)
 create mode 100644 data/staged/test_staged_v1_final.csv.dvc
 create mode 100644 data/staged/train_staged_v1_final.csv.dvc

✅ DVC push para remote:
2 files pushed

🚩 PROCESSO FINALIZADO — STAGED V1 FINAL VERSIONADO COM PYTHON PURO E CAMINHO CORRETO.


## Verificação Inicial e Carregamento do STAGED V1 FINAL

Nesta etapa damos início à camada CURATED V1, seguindo rigorosamente o Protocolo V5.4.  
A meta é garantir que o ambiente de trabalho esteja correto, os arquivos versionados existam fisicamente e possam ser carregados de forma íntegra. Essa validação evita falhas de caminho ou working directory que já comprometeram execuções anteriores.

Ao confirmar a existência dos arquivos, validamos o shape de cada conjunto e exibimos uma amostra para auditoria.  
Essa prática assegura rastreabilidade completa, criando uma base consistente para o pipeline de Feature Engineering Controlada que reduzirá a cardinalidade para fitting local viável.


In [22]:
# ETAPA: VERIFICACAO E CARREGAMENTO INICIAL STAGED V1 FINAL

import os
import pandas as pd
from tqdm import tqdm

# Caminhos
train_path = '/workspace/data/staged/train_staged_v1_final.csv'
test_path  = '/workspace/data/staged/test_staged_v1_final.csv'

# Verificação de CWD
print("\nDiretório de trabalho atual:", os.getcwd())

# Verificação de existência física
print("\nVerificando existência física dos arquivos:")
print("TRAIN:", os.path.exists(train_path))
print("TEST :", os.path.exists(test_path))

if not os.path.exists(train_path) or not os.path.exists(test_path):
    raise FileNotFoundError("Um dos arquivos STAGED V1 FINAL não foi encontrado.")

# Carregamento com tqdm
print("\nCarregando arquivos com pandas:")
train_df = pd.read_csv(train_path)
test_df  = pd.read_csv(test_path)

print("\nShapes verificados:")
print("TRAIN:", train_df.shape)
print("TEST :", test_df.shape)

# Amostra para auditoria
print("\nAmostra TRAIN:")
print(train_df.head(20))
print("\nAmostra TEST:")
print(test_df.head(20))



Diretório de trabalho atual: /workspace/notebooks

Verificando existência física dos arquivos:
TRAIN: True
TEST : True

Carregando arquivos com pandas:

Shapes verificados:
TRAIN: (100000, 46)
TEST : (50000, 45)

Amostra TRAIN:
       Month   Age Occupation  Annual_Income  Monthly_Inhand_Salary  \
0    January  23.0  Scientist       19114.12            1824.843333   
1   February  23.0  Scientist       19114.12            3260.465000   
2      March  33.0  Scientist       19114.12            3260.465000   
3      April  23.0  Scientist       19114.12            3260.465000   
4        May  23.0  Scientist       19114.12            1824.843333   
5       June  23.0  Scientist       19114.12            3260.465000   
6       July  23.0  Scientist       19114.12            1824.843333   
7     August  23.0  Scientist       19114.12            1824.843333   
8    January  33.0      Other       34847.84            3037.986667   
9   February  28.0    Teacher       34847.84            3037.

## Diagnóstico de Cardinalidade das Variáveis

Esta etapa tem como objetivo mapear todas as colunas do conjunto `TRAIN` para identificar o número de valores únicos de cada variável.  
Essa análise é fundamental para o desenho de estratégias de redução de cardinalidade, como o agrupamento de categorias raras e a aplicação de binning supervisionado.

O resultado será exportado em formato CSV para auditoria e versionamento.  
Essa prática garante rastreabilidade do diagnóstico e apoio a decisões futuras no pipeline de Feature Engineering Controlada.


In [23]:
# ETAPA: DIAGNOSTICO DE CARDINALIDADE

import pandas as pd
from tqdm import tqdm

# Geração do relatório de cardinalidade
cardinality_report = []

print("\nIniciando diagnóstico de cardinalidade...")
for col in tqdm(train_df.columns):
    unique_vals = train_df[col].nunique(dropna=False)
    cardinality_report.append((col, unique_vals))

# Cria DataFrame de saída
cardinality_df = pd.DataFrame(cardinality_report, columns=['Coluna', 'Valores_Unicos'])
cardinality_df = cardinality_df.sort_values(by='Valores_Unicos', ascending=False).reset_index(drop=True)

print("\nRelatório de cardinalidade:")
print(cardinality_df)

# Exporta para CSV
output_path = '/workspace/data/curated/cardinality_diagnosis_v1.csv'
os.makedirs('/workspace/data/curated', exist_ok=True)
cardinality_df.to_csv(output_path, index=False)
print(f"\nRelatório salvo em: {output_path}")



Iniciando diagnóstico de cardinalidade...


100%|██████████| 46/46 [00:00<00:00, 1302.22it/s]


Relatório de cardinalidade:
                                           Coluna  Valores_Unicos
0                        Credit_Utilization_Ratio          100000
1                                 Monthly_Balance           98794
2                         Amount_invested_monthly           91058
3                             Total_EMI_per_month           14950
4                                   Annual_Income           13439
5                           Monthly_Inhand_Salary           13238
6                                Outstanding_Debt           12204
7                                    Type_of_Loan            6261
8                            Changed_Credit_Limit            4377
9                                   Interest_Rate            1750
10                                            Age            1660
11                           Num_Credit_Inquiries            1224
12                                Num_Credit_Card            1179
13                              Num_Bank_Accoun

## Diagnóstico Exploratória Detalhada das Variáveis com Alta Cardinalidade

Nesta etapa, aprofundamos a análise das **20 colunas com maior cardinalidade** para compreender a distribuição dos dados e embasar decisões de binning ou agrupamento.  
Para cada coluna, serão calculados estatísticos resumidos, como mínimo, máximo, média, mediana, desvio padrão e percentis principais.

Esse diagnóstico garante que todo critério de transformação tenha **coerência com o negócio** e seja **interpretável**, respeitando o Protocolo V5.4 de rastreabilidade e justificativa técnica.
O resultado será exibido em uma **tabela única formatada**, dentro do próprio notebook, facilitando a análise interativa.


In [24]:
# ETAPA: DIAGNOSTICO EXPLORATORIO DETALHADO DAS TOP 20 VARIAVEIS

import pandas as pd
import numpy as np

# Seleciona as 20 colunas com maior cardinalidade
top20_cols = cardinality_df.head(20)['Coluna'].tolist()

# Lista para armazenar resultados
exploration_data = []

for col in top20_cols:
    serie = train_df[col]
    if pd.api.types.is_numeric_dtype(serie):
        summary = {
            'Coluna': col,
            'Tipo': 'Numerica',
            'Valores_Unicos': serie.nunique(dropna=False),
            'Min': serie.min(),
            'Max': serie.max(),
            'Mean': serie.mean(),
            'Median': serie.median(),
            'Std': serie.std(),
            'P5': serie.quantile(0.05),
            'P25': serie.quantile(0.25),
            'P50': serie.quantile(0.50),
            'P75': serie.quantile(0.75),
            'P95': serie.quantile(0.95)
        }
    else:
        freq = serie.value_counts(dropna=False).head(5)
        summary = {
            'Coluna': col,
            'Tipo': 'Categórica',
            'Valores_Unicos': serie.nunique(dropna=False),
            'Mais_Comum_1': freq.index[0] if len(freq) > 0 else None,
            'Freq_1': freq.iloc[0] if len(freq) > 0 else None,
            'Mais_Comum_2': freq.index[1] if len(freq) > 1 else None,
            'Freq_2': freq.iloc[1] if len(freq) > 1 else None,
            'Mais_Comum_3': freq.index[2] if len(freq) > 2 else None,
            'Freq_3': freq.iloc[2] if len(freq) > 2 else None
        }
    exploration_data.append(summary)

# Converte para DataFrame
exploration_df = pd.DataFrame(exploration_data)

# Exibe tabela formatada no notebook
from IPython.display import display
display(exploration_df)

# Salva para auditoria
output_path = '/workspace/data/curated/cardinality_exploration_top20.csv'
exploration_df.to_csv(output_path, index=False)
print(f"Diagnóstico detalhado salvo em: {output_path}")


,Coluna,Tipo,Valores_Unicos,Min,Max,Mean,Median,Std,P5,P25,P50,P75,P95,Mais_Comum_1,Freq_1,Mais_Comum_2,Freq_2,Mais_Comum_3,Freq_3
0,Credit_Utilization_Ratio,Numerica,100000,20.000000,5.000000e+01,32.285173,32.305784,5.116875e+00,24.230834,28.052567,32.305784,36.496663,40.220207,NaN,NaN,NaN,NaN,NaN,NaN
1,Monthly_Balance,Numerica,98794,0.007760,1.602041e+03,401.757116,336.762318,2.127499e+02,175.607996,270.913865,336.762318,467.670597,860.442449,NaN,NaN,NaN,NaN,NaN,NaN
2,Amount_invested_monthly,Numerica,91058,0.000000,1.977326e+03,189.695054,129.035357,1.915280e+02,32.523132,77.017414,129.035357,220.039055,588.692391,NaN,NaN,NaN,NaN,NaN,NaN
3,Total_EMI_per_month,Numerica,14950,0.000000,8.233100e+04,1403.118217,69.249473,8.306041e+03,0.000000,30.306660,69.249473,161.224249,437.012753,NaN,NaN,NaN,NaN,NaN,NaN
4,Annual_Income,Numerica,13439,7005.930000,2.419806e+07,168737.251723,37616.900000,1.392074e+06,9898.815000,20062.860000,37616.900000,70064.920000,132606.690000,NaN,NaN,NaN,NaN,NaN,NaN
5,Monthly_Inhand_Salary,Numerica,13238,303.645417,1.520463e+04,4030.768674,3116.892500,2.961116e+03,888.563750,1792.084167,3116.892500,5371.525000,10474.431000,NaN,NaN,NaN,NaN,NaN,NaN
6,Outstanding_Debt,Numerica,12204,0.230000,4.998070e+03,1423.862010,1167.200000,1.149508e+03,121.020000,571.650000,1167.200000,1933.430000,4071.620000,NaN,NaN,NaN,NaN,NaN,NaN
7,Type_of_Loan,Categórica,6261,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11408.0,Not Specified,1408.0,Credit-Builder Loan,1280.0
8,Changed_Credit_Limit,Numerica,4377,-6.490000,3.697000e+01,10.368671,9.400000,6.719636e+00,1.180000,5.420000,9.400000,14.660000,23.480000,NaN,NaN,NaN,NaN,NaN,NaN
9,Interest_Rate,Numerica,1750,1.000000,5.797000e+03,72.466040,13.000000,4.664226e+02,2.000000,8.000000,13.000000,20.000000,33.000000,NaN,NaN,NaN,NaN,NaN,NaN


Diagnóstico detalhado salvo em: /workspace/data/curated/cardinality_exploration_top20.csv


## Aplicação Completa de Binning Supervisionado nas Variáveis de Alta Cardinalidade

Com base no diagnóstico exploratório e nos percentis analisados, esta etapa aplica o **binning supervisionado** para as variáveis numéricas de maior cardinalidade.  
Os cortes são definidos com coerência de negócio, garantindo interpretabilidade e redução significativa de cardinalidade, viabilizando fitting local.

Todas as novas faixas serão criadas como colunas adicionais com sufixo `_Binned`, seguindo naming coerente.  
O DataFrame resultante manterá as variáveis originais para rastreabilidade e será salvo na pasta `data/curated/`, versionado posteriormente via DVC.


## Aplicação Completa do Binning Supervisionado e Agrupamentos para as Top 20 Variáveis

Nesta etapa, aplicamos todas as transformações supervisionadas nas variáveis de maior cardinalidade, conforme os percentis e o contexto de negócio definidos.  
Serão criados cortes com faixas interpretáveis, merge de categorias equivalentes e ajustes para inconsistências, garantindo fitting viável sem perda de rastreabilidade.

A variável `Credit_History_Age` será convertida de string para meses antes de aplicar binning.  
A coluna `Occupation` será normalizada para mesclar `Others` e `Outros` em um único grupo, além de consolidar ocupações raras como **“Outros”**.  
As variáveis `Month` e `Payment_Behaviour` permanecem inalteradas.

Todos os novos campos terão sufixo `_Binned` ou `_Group` e o resultado final será salvo na camada `CURATED`.


In [25]:
# ETAPA: BINNING SUPERVISIONADO E AGRUPAMENTOS TOP 20

import pandas as pd
import numpy as np

# Função auxiliar para Credit_History_Age
def parse_credit_history_age(value):
    try:
        parts = value.split(' ')
        years = int(parts[0])
        months = int(parts[3])
        return years * 12 + months
    except:
        return np.nan

# Credit_History_Age convertido para meses
train_df['Credit_History_Age_Months'] = train_df['Credit_History_Age'].apply(parse_credit_history_age)
train_df['Credit_History_Age_Binned'] = pd.cut(
    train_df['Credit_History_Age_Months'],
    bins=[-0.1, 24, 60, float('inf')],
    labels=['Curto', 'Médio', 'Longo']
)

# Credit_Utilization_Ratio
train_df['Credit_Utilization_Ratio_Binned'] = pd.cut(
    train_df['Credit_Utilization_Ratio'],
    bins=[0, 25, 35, float('inf')],
    labels=['Baixo', 'Moderado', 'Alto']
)

# Monthly_Balance
train_df['Monthly_Balance_Binned'] = pd.cut(
    train_df['Monthly_Balance'],
    bins=[0, 250, 500, float('inf')],
    labels=['Baixo', 'Moderado', 'Alto']
)

# Amount_invested_monthly
train_df['Amount_invested_monthly_Binned'] = pd.cut(
    train_df['Amount_invested_monthly'],
    bins=[-0.1, 50, 200, 600, float('inf')],
    labels=['Nenhum', 'Baixo', 'Moderado', 'Alto']
)

# Total_EMI_per_month
train_df['Total_EMI_per_month_Binned'] = pd.cut(
    train_df['Total_EMI_per_month'],
    bins=[-0.1, 0.1, 200, float('inf')],
    labels=['Sem_EMI', 'Baixo', 'Alto']
)

# Annual_Income
train_df['Annual_Income_Binned'] = pd.cut(
    train_df['Annual_Income'],
    bins=[0, 20000, 70000, 150000, float('inf')],
    labels=['Baixa', 'Média', 'Alta', 'Muito_Alta']
)

# Monthly_Inhand_Salary
train_df['Monthly_Inhand_Salary_Binned'] = pd.cut(
    train_df['Monthly_Inhand_Salary'],
    bins=[0, 1500, 5000, float('inf')],
    labels=['Baixo', 'Moderado', 'Alto']
)

# Outstanding_Debt
train_df['Outstanding_Debt_Binned'] = pd.cut(
    train_df['Outstanding_Debt'],
    bins=[0, 500, 2000, float('inf')],
    labels=['Baixo', 'Moderado', 'Alto']
)

# Changed_Credit_Limit
train_df['Changed_Credit_Limit_Binned'] = pd.cut(
    train_df['Changed_Credit_Limit'],
    bins=[-float('inf'), 0, 0.01, 10, float('inf')],
    labels=['Redução', 'Sem_Mudança', 'Aumento_Baixo', 'Aumento_Alto']
)

# Interest_Rate
train_df['Interest_Rate_Binned'] = pd.cut(
    train_df['Interest_Rate'],
    bins=[0, 20, 50, float('inf')],
    labels=['Normal', 'Acima_Padrao', 'Outlier']
)

# Age
train_df['Age_Binned'] = pd.cut(
    train_df['Age'],
    bins=[0, 25, 50, 100, float('inf')],
    labels=['Jovem', 'Adulto', 'Idoso', 'Erro']
)

# Num_Credit_Inquiries
train_df['Num_Credit_Inquiries_Binned'] = pd.cut(
    train_df['Num_Credit_Inquiries'],
    bins=[-0.1, 0.1, 5, 10, float('inf')],
    labels=['Nenhuma', 'Poucas', 'Média', 'Muitas']
)

# Num_Credit_Card
train_df['Num_Credit_Card_Binned'] = pd.cut(
    train_df['Num_Credit_Card'],
    bins=[-0.1, 0.1, 3, 7, float('inf')],
    labels=['Nenhum', '1-3', '4-7', '8+']
)

# Num_Bank_Accounts
train_df['Num_Bank_Accounts_Binned'] = pd.cut(
    train_df['Num_Bank_Accounts'],
    bins=[-0.1, 0.1, 2, 5, float('inf')],
    labels=['Nenhuma', '1-2', '3-5', '6+']
)

# Num_of_Delayed_Payment
train_df['Num_of_Delayed_Payment_Binned'] = pd.cut(
    train_df['Num_of_Delayed_Payment'],
    bins=[-float('inf'), -0.1, 0.1, 5, float('inf')],
    labels=['Erro_Neg', 'Nenhum', 'Poucos', 'Muitos']
)

# Num_of_Loan
train_df['Num_of_Loan_Binned'] = pd.cut(
    train_df['Num_of_Loan'],
    bins=[-float('inf'), -0.1, 0.1, 3, float('inf')],
    labels=['Erro_Neg', 'Nenhum', '1-3', '4+']
)

# Delay_from_due_date
train_df['Delay_from_due_date_Binned'] = pd.cut(
    train_df['Delay_from_due_date'],
    bins=[-float('inf'), -0.1, 0.1, 10, 30, float('inf')],
    labels=['Antecipado', 'No_Prazo', 'Pouco', 'Moderado', 'Critico']
)

# Occupation: normaliza Others/Outros e agrupa ocupações raras
train_df['Occupation_Group'] = train_df['Occupation'].replace({'Others': 'Outros'})
freq = train_df['Occupation_Group'].value_counts(normalize=True)
rare_labels = freq[freq < 0.05].index
train_df['Occupation_Group'] = train_df['Occupation_Group'].apply(lambda x: 'Outros' if x in rare_labels else x)

# Salva resultado final
output_path = '/workspace/data/curated/train_curated_v1.csv'
train_df.to_csv(output_path, index=False)
print(f'Arquivo CURATED V1 salvo em: {output_path}')


Arquivo CURATED V1 salvo em: /workspace/data/curated/train_curated_v1.csv


## Aplicação Completa do Binning e Agrupamentos no Conjunto TEST

Esta etapa replica todas as transformações supervisionadas e agrupamentos do `TRAIN` no conjunto `TEST`, garantindo **consistência de features** entre treino e teste.  
As mesmas faixas de binning, parsing de `Credit_History_Age` e merge de categorias para `Occupation` são aplicados sem exceção.

O arquivo final será salvo na camada `CURATED` e preparado para versionamento coerente.


In [26]:
# ETAPA: BINNING SUPERVISIONADO E AGRUPAMENTOS NO TEST

# Função auxiliar para Credit_History_Age
def parse_credit_history_age(value):
    try:
        parts = value.split(' ')
        years = int(parts[0])
        months = int(parts[3])
        return years * 12 + months
    except:
        return np.nan

# Credit_History_Age
test_df['Credit_History_Age_Months'] = test_df['Credit_History_Age'].apply(parse_credit_history_age)
test_df['Credit_History_Age_Binned'] = pd.cut(
    test_df['Credit_History_Age_Months'],
    bins=[-0.1, 24, 60, float('inf')],
    labels=['Curto', 'Médio', 'Longo']
)

# Credit_Utilization_Ratio
test_df['Credit_Utilization_Ratio_Binned'] = pd.cut(
    test_df['Credit_Utilization_Ratio'],
    bins=[0, 25, 35, float('inf')],
    labels=['Baixo', 'Moderado', 'Alto']
)

# Monthly_Balance
test_df['Monthly_Balance_Binned'] = pd.cut(
    test_df['Monthly_Balance'],
    bins=[0, 250, 500, float('inf')],
    labels=['Baixo', 'Moderado', 'Alto']
)

# Amount_invested_monthly
test_df['Amount_invested_monthly_Binned'] = pd.cut(
    test_df['Amount_invested_monthly'],
    bins=[-0.1, 50, 200, 600, float('inf')],
    labels=['Nenhum', 'Baixo', 'Moderado', 'Alto']
)

# Total_EMI_per_month
test_df['Total_EMI_per_month_Binned'] = pd.cut(
    test_df['Total_EMI_per_month'],
    bins=[-0.1, 0.1, 200, float('inf')],
    labels=['Sem_EMI', 'Baixo', 'Alto']
)

# Annual_Income
test_df['Annual_Income_Binned'] = pd.cut(
    test_df['Annual_Income'],
    bins=[0, 20000, 70000, 150000, float('inf')],
    labels=['Baixa', 'Média', 'Alta', 'Muito_Alta']
)

# Monthly_Inhand_Salary
test_df['Monthly_Inhand_Salary_Binned'] = pd.cut(
    test_df['Monthly_Inhand_Salary'],
    bins=[0, 1500, 5000, float('inf')],
    labels=['Baixo', 'Moderado', 'Alto']
)

# Outstanding_Debt
test_df['Outstanding_Debt_Binned'] = pd.cut(
    test_df['Outstanding_Debt'],
    bins=[0, 500, 2000, float('inf')],
    labels=['Baixo', 'Moderado', 'Alto']
)

# Changed_Credit_Limit
test_df['Changed_Credit_Limit_Binned'] = pd.cut(
    test_df['Changed_Credit_Limit'],
    bins=[-float('inf'), 0, 0.01, 10, float('inf')],
    labels=['Redução', 'Sem_Mudança', 'Aumento_Baixo', 'Aumento_Alto']
)

# Interest_Rate
test_df['Interest_Rate_Binned'] = pd.cut(
    test_df['Interest_Rate'],
    bins=[0, 20, 50, float('inf')],
    labels=['Normal', 'Acima_Padrao', 'Outlier']
)

# Age
test_df['Age_Binned'] = pd.cut(
    test_df['Age'],
    bins=[0, 25, 50, 100, float('inf')],
    labels=['Jovem', 'Adulto', 'Idoso', 'Erro']
)

# Num_Credit_Inquiries
test_df['Num_Credit_Inquiries_Binned'] = pd.cut(
    test_df['Num_Credit_Inquiries'],
    bins=[-0.1, 0.1, 5, 10, float('inf')],
    labels=['Nenhuma', 'Poucas', 'Média', 'Muitas']
)

# Num_Credit_Card
test_df['Num_Credit_Card_Binned'] = pd.cut(
    test_df['Num_Credit_Card'],
    bins=[-0.1, 0.1, 3, 7, float('inf')],
    labels=['Nenhum', '1-3', '4-7', '8+']
)

# Num_Bank_Accounts
test_df['Num_Bank_Accounts_Binned'] = pd.cut(
    test_df['Num_Bank_Accounts'],
    bins=[-0.1, 0.1, 2, 5, float('inf')],
    labels=['Nenhuma', '1-2', '3-5', '6+']
)

# Num_of_Delayed_Payment
test_df['Num_of_Delayed_Payment_Binned'] = pd.cut(
    test_df['Num_of_Delayed_Payment'],
    bins=[-float('inf'), -0.1, 0.1, 5, float('inf')],
    labels=['Erro_Neg', 'Nenhum', 'Poucos', 'Muitos']
)

# Num_of_Loan
test_df['Num_of_Loan_Binned'] = pd.cut(
    test_df['Num_of_Loan'],
    bins=[-float('inf'), -0.1, 0.1, 3, float('inf')],
    labels=['Erro_Neg', 'Nenhum', '1-3', '4+']
)

# Delay_from_due_date
test_df['Delay_from_due_date_Binned'] = pd.cut(
    test_df['Delay_from_due_date'],
    bins=[-float('inf'), -0.1, 0.1, 10, 30, float('inf')],
    labels=['Antecipado', 'No_Prazo', 'Pouco', 'Moderado', 'Critico']
)

# Occupation: normaliza Others/Outros e agrupa ocupações raras
test_df['Occupation_Group'] = test_df['Occupation'].replace({'Others': 'Outros'})
freq_test = test_df['Occupation_Group'].value_counts(normalize=True)
rare_labels_test = freq_test[freq_test < 0.05].index
test_df['Occupation_Group'] = test_df['Occupation_Group'].apply(lambda x: 'Outros' if x in rare_labels_test else x)

# Salva resultado final
output_path_test = '/workspace/data/curated/test_curated_v1.csv'
test_df.to_csv(output_path_test, index=False)
print(f'Arquivo CURATED V1 TEST salvo em: {output_path_test}')


Arquivo CURATED V1 TEST salvo em: /workspace/data/curated/test_curated_v1.csv


## Versionamento Atômico da Camada CURATED V1

Nesta etapa aplicamos o **versionamento atômico** para os arquivos `train_curated_v1.csv` e `test_curated_v1.csv`.  
A ação inclui verificação de existência física, `dvc add`, `git add` dos metadados, commit coerente e push dos dados para o backend MinIO, garantindo rastreabilidade completa do snapshot CURATED V1.

Este fluxo cumpre integralmente o **PROTOCOLO V5.4**, com controle de diretório de trabalho (`CWD`) e auditoria de consistência.


In [27]:
# ETAPA: VERSIONAMENTO ATÔMICO DO CURATED V1

import os
import subprocess

# Caminhos reais dos arquivos
train_curated = '/workspace/data/curated/train_curated_v1.csv'
test_curated  = '/workspace/data/curated/test_curated_v1.csv'

# Verifica CWD
print("\nDiretório de trabalho atual:", os.getcwd())

# Confirma existência física
print("\nVerificando existência física:")
print("TRAIN CURATED:", os.path.exists(train_curated))
print("TEST CURATED :", os.path.exists(test_curated))

if not os.path.exists(train_curated) or not os.path.exists(test_curated):
    raise FileNotFoundError("Um dos arquivos CURATED V1 não foi encontrado.")

# DVC add
print("\nExecutando dvc add ...")
subprocess.run(['dvc', 'add', train_curated], check=True)
subprocess.run(['dvc', 'add', test_curated], check=True)

# Git add dos metadados .dvc
print("\nAdicionando metadados .dvc ao Git ...")
subprocess.run(['git', 'add', f"{train_curated}.dvc"], check=True)
subprocess.run(['git', 'add', f"{test_curated}.dvc"], check=True)

# Commit coerente
print("\nRealizando commit Git ...")
subprocess.run(['git', 'commit', '-m', 'Versionamento atômico CURATED V1: train_curated_v1.csv e test_curated_v1.csv'], check=True)

# DVC push para MinIO
print("\nExecutando dvc push ...")
subprocess.run(['dvc', 'push'], check=True)

# Git push final
print("\nExecutando git push ...")
subprocess.run(['git', 'push'], check=True)

print("\nVersionamento CURATED V1 concluído com sucesso.")



Diretório de trabalho atual: /workspace/notebooks

Verificando existência física:
TRAIN CURATED: True
TEST CURATED : True

Executando dvc add ...


⠋ Checking graph
⠋ Checking graph



Adicionando metadados .dvc ao Git ...

Realizando commit Git ...
[main 46ad007] Versionamento atômico CURATED V1: train_curated_v1.csv e test_curated_v1.csv
 2 files changed, 7 insertions(+), 2 deletions(-)
 create mode 100644 data/curated/test_curated_v1.csv.dvc

Executando dvc push ...
2 files pushed

Executando git push ...

Versionamento CURATED V1 concluído com sucesso.


To github.com:WRMELO/MBA_MLOPS.git
   9fae4d2..46ad007  main -> main
